# **1. Estrutura da Tabela**

#### **1. Visualizando a Estrutura Atual do Schema**
Antes de alterar qualquer coisa, é importante conhecer a estrutura atual da tabela. Você pode usar o comando PRAGMA para isso.



**Criando a tabela:**

In [ ]:
import sqlite3

conn = sqlite3.connect("../login.db")
cursor = conn.cursor()

# 1. Crie a nova tabela com a estrutura desejada.
cursor.execute("""
    CREATE TABLE IF NOT EXISTS users (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        name TEXT NOT NULL,
        username TEXT NOT NULL UNIQUE,
        password TEXT NOT NULL,
        role TEXT NOT NULL DEFAULT 'user' CHECK(role IN ('admin', 'superadmin', 'user'))
    );
""")
conn.commit()


In [ ]:
import sqlite3

def mostrar_estrutura(tabela, conn):
    cursor = conn.cursor()
    cursor.execute(f"PRAGMA table_info({tabela});")
    colunas = cursor.fetchall()
    for coluna in colunas:
        # Cada coluna retorna uma tupla: (cid, name, type, notnull, dflt_value, pk)
        print(coluna)

conn = sqlite3.connect("../login.db")
mostrar_estrutura("users", conn)
conn.close()


#### **2. Adicionando Novos Campos (Colunas)**
SQLite permite a adição de novas colunas utilizando o comando ALTER TABLE {name} ADD COLUMN {name}

obs.: A nova coluna será adicionada com valor NULL para as linhas já existentes


In [ ]:
conn = sqlite3.connect("../login.db")
cursor = conn.cursor()

# Exemplo: Adicionar a coluna "last_login" do tipo TEXT para armazenar a data/hora do último login.
cursor.execute("ALTER TABLE users ADD COLUMN last_login TEXT;")
conn.commit()
conn.close()


#### **3. Renomeando Tabelas e Colunas**
Renomear Tabela
SQLite permite renomear uma tabela diretamente:

In [ ]:
conn = sqlite3.connect("../login.db")
cursor = conn.cursor()

# Renomear a tabela "users" para "usuarios"
cursor.execute("ALTER TABLE users RENAME TO usuarios;")
conn.commit()
conn.close()


Renomear Coluna
A partir do SQLite 3.25.0 (lançado em 2018), é possível renomear uma coluna:

In [ ]:
conn = sqlite3.connect("../login.db")
cursor = conn.cursor()

# Exemplo: renomear a coluna "username" para "user_name" na tabela "usuarios"
cursor.execute("ALTER TABLE users RENAME COLUMN user_name TO username;")
conn.commit()
conn.close()


#### **4. Alterando Tipos de Dados ou Excluindo Colunas (Recriação de Tabela)**
SQLite não permite alterar diretamente o tipo de uma coluna ou remover uma coluna com comandos simples (até versões recentes, a partir da 3.35.0 há suporte a DROP COLUMN).

Para alterar o tipo de uma coluna ou remover uma coluna, a estratégia comum é:

1. Criar uma nova tabela com a estrutura desejada

2. Copiar os dados da tabela antiga para a nova.

3. Excluir a tabela antiga.

4. Renomear a nova tabela para o nome original.

**Exemplo: Alterando Tipo de Dados e Removendo uma Coluna**

Suponha que você queira alterar o tipo da coluna "last_login" para DATETIME (ou formatá-la melhor) e remover uma coluna chamada "campo_indesejado". Veja um exemplo:


In [ ]:
# 1. criando um novo banco de dados
import sqlite3

conn = sqlite3.connect("../login.db")
cursor = conn.cursor()

# 1. Crie a nova tabela com a estrutura desejada.
cursor.execute("""
    CREATE TABLE IF NOT EXISTS usuarios_nova (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        name TEXT NOT NULL,
        user_name TEXT NOT NULL UNIQUE,
        password TEXT NOT NULL,
        role TEXT NOT NULL DEFAULT 'user',
        last_login DATETIME  -- novo tipo desejado
        -- Note que 'campo_indesejado' não é incluído
    );
""")
conn.commit()


In [ ]:

# 2. Copie os dados da tabela antiga para a nova.
# Supondo que a tabela antiga seja "usuarios" e possua as colunas: id, name, user_name, password, role, last_login, campo_indesejado
cursor.execute("""
    INSERT INTO usuarios_nova (id, name, user_name, password, role, last_login)
    SELECT id, name, user_name, password, role, last_login
    FROM usuarios;
""")
conn.commit()


In [ ]:

# 3. Exclua a tabela antiga.
cursor.execute("DROP TABLE usuarios;")
conn.commit()


In [ ]:

# 4. Renomeie a nova tabela para o nome original.
cursor.execute("ALTER TABLE usuarios_nova RENAME TO usuarios;")
conn.commit()

conn.close()


# **2. Trabalhando com os dados**


In [11]:
#0 . criando uma conexão com o banco de dados
import sqlite3

def conectar_db(db_name='../login.db'):
    conn = sqlite3.connect(db_name)
    conn.row_factory = sqlite3.Row
    return conn
conn = conectar_db()  # Establish the connection


In [12]:
# 1. buscar usuarios
def buscar_usuarios(conn):
    cursor = conn.cursor()
    cursor.execute("SELECT * FROM users")
    return cursor.fetchall()

usuarios = buscar_usuarios(conn)
for user in usuarios:
    print(f"ID: {user['id']}, Nome: {user['name']}, Username: {user['username']}, Role: {user['role']}")

conn.close() 


ID: 1, Nome: maelson, Username: lima, Role: superadmin
ID: 4, Nome: maria, Username: silva, Role: user
ID: 5, Nome: maelson, Username: marques, Role: admin


In [ ]:
#2. criar uma tabela

def criar_tabela(conn):
    cursor = conn.cursor()
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS users (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            name TEXT NOT NULL,
            username TEXT NOT NULL UNIQUE,
            password TEXT NOT NULL,
            role TEXT NOT NULL DEFAULT 'user'
        )
    ''')
    conn.commit()


In [ ]:

def inserir_usuario(conn, name, username, password, role='user'):
    cursor = conn.cursor()
    cursor.execute('''
        INSERT INTO users (name, username, password, role)
        VALUES (?, ?, ?, ?)
    ''', (name, username, password, role))
    conn.commit()

def buscar_usuarios(conn):
    cursor = conn.cursor()
    cursor.execute("SELECT * FROM users")
    return cursor.fetchall()

def atualizar_usuario(conn, username, novo_nome):
    cursor = conn.cursor()
    cursor.execute('''
        UPDATE users
        SET name = ?
        WHERE username = ?
    ''', (novo_nome, username))
    conn.commit()

def remover_usuario(conn, username):
    cursor = conn.cursor()
    cursor.execute("DELETE FROM users WHERE username = ?", (username,))
    conn.commit()

# Uso do tutorial
conn = conectar_db()
criar_tabela(conn)

# Inserindo usuário
inserir_usuario(conn, "Alice Smith", "alice", "hash123", "user")
inserir_usuario(conn, "Bob Johnson", "bob", "hash456", "superadmin")


In [ ]:

# Buscando e exibindo usuários
usuarios = buscar_usuarios(conn)
for user in usuarios:
    print(f"ID: {user['id']}, Nome: {user['name']}, Username: {user['username']}, Role: {user['role']}")


In [ ]:

# Atualizando usuário
atualizar_usuario(conn, "alice", "Alice S.")


In [ ]:

# Removendo um usuário
remover_usuario(conn, "bob")

# Exibe os usuários restantes
print("Após remoção:")
usuarios = buscar_usuarios(conn)
for user in usuarios:
    print(dict(user))

conn.close()
